In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

In [2]:
Ridership_df= pd.read_csv('data/Ridership.csv')
df= Ridership_df.copy()
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
54654,2022,June,2,22,Corridor_7,y,Station_24,Midday,260,4,0
32081,2020,December,14,51,Corridor_3,y,Station_3,Evening,300,8,1
24628,2020,May,29,22,Corridor_4,y,Station_11,PM Peak,22,2,1
7136,2019,May,24,21,Corridor_4,y,Station_11,PM Peak,180,2,0
6805,2019,May,16,20,Corridor_2,y,Station_3,PM Peak,10188,12,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         64369 non-null  int64 
 1   Month        64369 non-null  object
 2   Day          64369 non-null  int64 
 3   Week Number  64369 non-null  int64 
 4   Corridor     64369 non-null  object
 5   Workday      64369 non-null  object
 6   Station      64369 non-null  object
 7   Period       64369 non-null  object
 8   Ridership    64369 non-null  int64 
 9   N_trains     64369 non-null  int64 
 10  Covid19      64369 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 5.4+ MB


In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.isnull().sum()

Year           0
Month          0
Day            0
Week Number    0
Corridor       0
Workday        0
Station        0
Period         0
Ridership      0
N_trains       0
Covid19        0
dtype: int64

In [4]:
month_mapping = {'January': 1, 'February':2, 'March': 3,
                 'April': 4, 'May': 5, 'June': 6, 'July': 7,
                 'August': 8, 'September': 9, 'October': 10, 
                 'November': 11, 'December': 12}


df['Month_Num'] = df['Month'].map(month_mapping)

In [5]:
def convert_day_to_circle(day):
    angle = 2 * np.pi * (day - 1) / 31  # Calculate the angle based on the day
    x = np.cos(angle)  # Compute the x-coordinate on the unit circle
    y = np.sin(angle)  # Compute the y-coordinate on the unit circle
    return x, y

def convert_week_to_circle(week):
    angle = 2 * np.pi * (week - 1) / 53  # Calculate the angle based on the week
    x = np.cos(angle)  # Compute the x-coordinate on the unit circle
    y = np.sin(angle)  # Compute the y-coordinate on the unit circle
    return x, y

df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))
df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))

In [6]:
categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

preprocessor = ColumnTransformer(
        transformers=[('', OneHotEncoder(), categorical_features)],
        remainder='passthrough')

df = preprocessor.fit_transform(df)
df = df.toarray()
feature_names_out = list(preprocessor.get_feature_names_out())
df= pd.DataFrame(df, columns=[item.split('__')[1] for item in feature_names_out])

In [7]:
df['COVID_Workday'] = df['Covid19'] * df['Workday_y']
columns_to_convert = [
    'Month_April', 'Month_August', 'Month_December', 'Month_February', 
    'Month_January', 'Month_July', 'Month_June', 'Month_March', 
    'Month_May', 'Month_November', 'Month_October', 'Month_September',
    'Corridor_Corridor_1', 'Corridor_Corridor_2', 'Corridor_Corridor_3', 
    'Corridor_Corridor_4', 'Corridor_Corridor_5', 'Corridor_Corridor_6', 'Corridor_Corridor_7',
    'Station_Station_1', 'Station_Station_10', 'Station_Station_11', 'Station_Station_12', 
    'Station_Station_13', 'Station_Station_14', 'Station_Station_15', 'Station_Station_16', 
    'Station_Station_17', 'Station_Station_18', 'Station_Station_19', 'Station_Station_2', 
    'Station_Station_20', 'Station_Station_21', 'Station_Station_22', 'Station_Station_23', 
    'Station_Station_24', 'Station_Station_25', 'Station_Station_26', 'Station_Station_27', 
    'Station_Station_28', 'Station_Station_29', 'Station_Station_3', 'Station_Station_30', 
    'Station_Station_31', 'Station_Station_32', 'Station_Station_33', 'Station_Station_34', 
    'Station_Station_35', 'Station_Station_36', 'Station_Station_37', 'Station_Station_38', 
    'Station_Station_39', 'Station_Station_4', 'Station_Station_40', 'Station_Station_41', 
    'Station_Station_42', 'Station_Station_43', 'Station_Station_44', 'Station_Station_45', 
    'Station_Station_5', 'Station_Station_6', 'Station_Station_7', 'Station_Station_8', 'Station_Station_9',
    'Period_AM Peak', 'Period_Evening', 'Period_Midday', 'Period_PM Peak', 'Period_Weekend/Holiday',
    'Day',
    'Week Number',
    'Workday_n', 'Workday_y',
    'Covid19','COVID_Workday',
    'Month_Num']

columns_to_convert_02=['Year', 'Ridership', 'N_trains']


df[columns_to_convert] = df[columns_to_convert].astype('uint8')
df[columns_to_convert_02] = df[columns_to_convert_02].astype('uint16')


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 83 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_April             64369 non-null  uint8  
 1   Month_August            64369 non-null  uint8  
 2   Month_December          64369 non-null  uint8  
 3   Month_February          64369 non-null  uint8  
 4   Month_January           64369 non-null  uint8  
 5   Month_July              64369 non-null  uint8  
 6   Month_June              64369 non-null  uint8  
 7   Month_March             64369 non-null  uint8  
 8   Month_May               64369 non-null  uint8  
 9   Month_November          64369 non-null  uint8  
 10  Month_October           64369 non-null  uint8  
 11  Month_September         64369 non-null  uint8  
 12  Corridor_Corridor_1     64369 non-null  uint8  
 13  Corridor_Corridor_2     64369 non-null  uint8  
 14  Corridor_Corridor_3     64369 non-null

In [11]:
df.sample(10)

,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,...,Week Number,Ridership,N_trains,Covid19,Month_Num,day_x,day_y,week_x,week_y,COVID_Workday
19836,0,0,0,1,0,0,0,0,0,0,...,6,394,1,1,2,0.528964,0.848644,0.829406,0.558647,1
17453,0,0,1,0,0,0,0,0,0,0,...,51,880,1,0,12,-0.874347,-0.485302,0.937420,-0.348202,0
61693,0,0,0,0,0,0,0,0,0,1,...,44,868,3,0,11,0.820763,0.571268,0.375828,-0.926690,0
44614,0,0,0,0,0,0,0,0,0,0,...,42,293,6,1,10,-0.440394,-0.897805,0.147647,-0.989040,1
43857,0,0,0,0,0,0,0,0,0,0,...,40,101,3,1,10,0.347305,0.937752,-0.088796,-0.996050,1
33038,0,0,0,0,1,0,0,0,0,0,...,1,151,8,1,1,0.347305,0.937752,1.000000,0.000000,1
33938,0,0,0,0,1,0,0,0,0,0,...,4,39,1,1,1,0.688967,-0.724793,0.937420,0.348202,1
46822,0,0,1,0,0,0,0,0,0,0,...,48,511,9,1,12,0.918958,0.394356,0.757511,-0.652822,1
27262,0,1,0,0,0,0,0,0,0,0,...,34,130,6,1,8,-0.954139,-0.299363,-0.717507,-0.696551,1
62854,0,0,0,0,0,0,0,0,0,1,...,48,99,1,0,11,0.918958,-0.394356,0.757511,-0.652822,0


In [8]:
def split_the_timeseries_data(Ridership_df, df):
    # Create empty lists to store the train-test split data
    X_train = []
    X_test = []
    y_train = []
    y_test = []

    # Iterate over each unique station
    for station in Ridership_df['Station'].unique():
        # Get the data for the current station
        station_data = df[df['Station_'+station] == 1]
        target_data = Ridership_df.loc[df['Station_'+station] == 1, 'Ridership']

        # Determine the number of samples to include in the test set
        test_size = int(len(station_data) * 0.2)

        # Split the data into train and test sets
        station_X_train = station_data[:-test_size]
        station_X_test = station_data[-test_size:]
        station_y_train = target_data[:-test_size]
        station_y_test = target_data[-test_size:]

        # Append the station-specific data to the overall train-test split
        X_train.append(station_X_train)
        X_test.append(station_X_test)
        y_train.append(station_y_train)
        y_test.append(station_y_test)

    # Concatenate the train-test split data
    X_train = pd.concat(X_train)
    X_test = pd.concat(X_test)
    y_train = pd.concat(y_train)
    y_test = pd.concat(y_test)

    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_the_timeseries_data(Ridership_df, df)

## Problem One: 
Assume we want to find a general model for the number of passengers without looking at the number of passengers on previous days and only using the recorded information in the table for the same time period (except for the year and number of trains) to predict the number of required trains based on the number of passengers.

In [9]:
X_train_p01= X_train.drop(['Ridership','Year','N_trains'],axis=1)
X_test_p01= X_test.drop(['Ridership','Year','N_trains'],axis=1)

y_train_p01= y_train.copy()
y_test_p01= y_test.copy()

In [99]:
svr_model= SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

svr_model.fit(X_train_p01, y_train_p01)

y_pred_svr_p01 = svr_model.predict(X_test_p01)

# Evaluate the model performance
rmse = np.sqrt(mean_squared_error(y_test_p01, y_pred_svr_p01))
r2 = r2_score(y_test_p01, y_pred_svr_p01)

print(f"\nModel Performance Evaluation:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.4f}")




Model Performance Evaluation:
Root Mean Squared Error (RMSE): 1244.84
R-squared (R²): 0.1874


In [25]:
y_pred_svr_p01 = pd.DataFrame(y_pred_svr_p01, columns=['Predicted_Passengers'])

train_capacity = 600  
y_pred_svr_p01['Predicted_Trains'] = np.ceil(y_pred_svr_p01['Predicted_Passengers'] / train_capacity).astype(int)

y_pred_svr_p01.sample(5)


,Predicted_Passengers,Predicted_Trains
10662,1527.114098,3
3555,1135.261578,2
777,503.079604,1
1831,879.806692,2
2679,1119.657423,2


In [17]:
param_grid_xgb = {
    'n_estimators': [50,100,200,300,400,500,600,700,800,900,1000],   
    'learning_rate': [0.1,0.2,0.3],
    'max_depth':  [3,5,7,9,10],
    'subsample':  [0.1,0.3,0.5,0.7,0.9,1],
    'colsample_bytree':  [0.5,0.7,0.9, 1],
    }

grid_xgb = GridSearchCV(XGBRegressor(objective='reg:squarederror'), param_grid_xgb)

grid_xgb.fit(X_train_p01, y_train_p01)

print(grid_xgb.best_params_)
print(grid_xgb.best_estimator_)

KeyboardInterrupt: 

In [97]:
X_train_xgb_p01 = X_train_p01.astype(float)
X_test_xgb_p01 = X_test_p01.astype(float)

xgb_model = XGBRegressor(objective='reg:squarederror',
                         n_estimators=36,
                         learning_rate=0.01,
                         max_depth=10,
                         subsample=0.9,
                         colsample_bytree=1,
                         random_state=42,
                         n_jobs=-1)


xgb_model.fit(X_train_xgb_p01, y_train_p01)

y_pred_xgb_p01 = xgb_model.predict(X_test_xgb_p01)

rmse_xgb = np.sqrt(mean_squared_error(y_test_p01, y_pred_xgb_p01))
r2_xgb = r2_score(y_test_p01, y_pred_xgb_p01)


print(f"Root Mean Squared Error (RMSE): {rmse_xgb:.2f}")
print(f"R-squared (R²): {r2_xgb:.4f}")

Root Mean Squared Error (RMSE): 1039.02
R-squared (R²): 0.4339
